# Feature Engineering en SQL

A continuación, veremos cómo calcular diferentes variables para el feature engineering utilizando SQL.


In [1]:
# %pip install duckdb
# %pip install jupysql
# %pip install duckdb-engine

In [2]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:default:

In [3]:
dataset_file = 'datos/competencia_01_brandoni.csv'

In [4]:
%%sql
create or replace table competencia_01 as
select
    *
from read_csv_auto("{{dataset_file}}")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


Definir la macro para sumar sin valores nulos


In [5]:
%%sql
CREATE OR REPLACE MACRO suma_sin_null(a, b) AS ifnull(a, 0) + ifnull(b, 0);

,Success


Crear nueva tabla con todos los atributos originales más los nuevos de ingeniería de variables

https://chatgpt.com/share/66edc29b-e0a4-8004-aa86-508d1e7eefa5


In [6]:
%%sql
CREATE or REPLACE TABLE competencia_01_fe as
SELECT
    *,

    suma_sin_null(Master_msaldototal, Visa_msaldototal) as tc_saldo_total,
    suma_sin_null(mcuenta_corriente, mcaja_ahorro) as saldo_total_cuentas,
    suma_sin_null(suma_sin_null(Visa_msaldototal, Master_msaldototal), suma_sin_null(mcuenta_corriente, mcaja_ahorro)) as saldo_total_completo,
    suma_sin_null(mprestamos_personales, suma_sin_null(mprestamos_prendarios, mprestamos_hipotecarios)) as deuda_total_prestamos,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY cliente_antiguedad) as cliente_antiguedad_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY mpayroll) as payroll_10tile,
    ntile(10) OVER (partition BY foto_mes ORDER BY mtarjeta_visa_consumo) as visa_consumo_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY ctarjeta_visa) as visa_tarjetas_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY mtarjeta_master_consumo) as master_consumo_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY ctarjeta_master) as master_tarjetas_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY cproductos) as productos_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY mcuenta_corriente) as cuenta_corriente_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY mcaja_ahorro) as caja_ahorro_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY suma_sin_null(ctarjeta_visa, ctarjeta_master)) as tarjetas_credito_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY suma_sin_null(mtarjeta_visa_consumo, mtarjeta_master_consumo)) as tarjetas_consumo_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY mprestamos_personales) as prestamos_personales_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY mprestamos_hipotecarios) as prestamos_hipotecarios_10tile,

    lag(mpayroll, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as lag_1_mpayroll,
    avg(mpayroll) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 3 PRECEDING AND CURRENT ROW) as avg_3_mpayroll,
    min(mpayroll) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 3 PRECEDING AND CURRENT ROW) as min_3_mpayroll,
    max(mpayroll) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 3 PRECEDING AND CURRENT ROW) as max_3_mpayroll,
    lag(suma_sin_null(mtarjeta_visa_consumo, mtarjeta_master_consumo), 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as lag_1_tarjetas_consumo,
    lag(mcuenta_corriente, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as lag_1_cuenta_corriente,
    lag(mcaja_ahorro, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as lag_1_caja_ahorro,

    avg(mcuenta_corriente) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 3 PRECEDING AND CURRENT ROW) as avg_3_cuenta_corriente,
    min(mprestamos_personales) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 3 PRECEDING AND CURRENT ROW) as min_3_prestamos_personales,
    max(mprestamos_personales) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 3 PRECEDING AND CURRENT ROW) as max_3_prestamos_personales,

    mpayroll - lag(mpayroll, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as delta_1_mpayroll,
    mtarjeta_visa_consumo - lag(mtarjeta_visa_consumo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as delta_1_visa_consumo,
    mcuenta_corriente - lag(mcuenta_corriente, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as delta_1_cuenta_corriente,
    mprestamos_personales - lag(mprestamos_personales, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as delta_1_prestamos_personales,
    ctarjeta_visa_transacciones - lag(ctarjeta_visa_transacciones, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as delta_1_transacciones_visa,

    avg(mpayroll) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 3 PRECEDING AND CURRENT ROW) as mov_avg_3_mpayroll,
    avg(mcuenta_corriente) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 3 PRECEDING AND CURRENT ROW) as mov_avg_3_cuenta_corriente,
    avg(mprestamos_personales) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 3 PRECEDING AND CURRENT ROW) as mov_avg_3_prestamos_personales,

    regr_slope(mpayroll, cliente_antiguedad) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 3 PRECEDING AND CURRENT ROW) as payroll_slope_3,
    regr_slope(mcuenta_corriente, cliente_antiguedad) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 3 PRECEDING AND CURRENT ROW) as cuenta_corriente_slope_3,
    regr_slope(mtarjeta_visa_consumo, cliente_antiguedad) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 3 PRECEDING AND CURRENT ROW) as visa_consumo_slope_3,

    case when Master_status in (6, 7, 9) then 1 else 0 end as Master_cuenta_cerrada,
    case when Visa_status in (6, 7, 9) then 1 else 0 end as Visa_cuenta_cerrada,
    mcuentas_saldo / nullif(suma_sin_null(mactivos_margen, mpasivos_margen), 0) as ratio_saldo_margen,
    suma_sin_null(mcomisiones_mantenimiento, mcomisiones_otras) as total_comisiones_cliente,

    suma_sin_null(ctarjeta_visa_transacciones, ctarjeta_master_transacciones) as transacciones_totales_tarjetas,
    ctarjeta_debito_transacciones - suma_sin_null(ctarjeta_visa_transacciones, ctarjeta_master_transacciones) as delta_debito_vs_credito,

    thomebanking + tmobile_app as indicador_banca_digital

FROM competencia_01


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


Y a no olvidarse guardar las nueva tabla

In [7]:
%%sql
COPY competencia_01_fe TO 'datos/competencia_01_brandoni_fe.csv' (FORMAT CSV, HEADER TRUE);

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


# PASAR a un FE agresivo